In [1]:
import requests
from bs4 import BeautifulSoup as bs

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

from tqdm import tqdm

import re
import pandas as pd

import free_proxy

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://lostark.inven.co.kr/'

res = requests.get(url)

In [3]:
class_list = bs(res.text, 'html.parser').select_one('#lostarkBody > div.commu-wrap > section > article > section.commu-left > div.left-menu > ul.list.tablist')

In [4]:
class_list = class_list.select('span > a')

In [126]:
import json

with open ('./data/class_doe.json', 'w') as f:
    json.dump(class_code,f )

In [5]:
class_code = dict()
class_url = dict()

for cl in class_list:
    url = cl['href']
    base_url = 'https://www.inven.co.kr/board/lostark/'
    code = int(re.sub(base_url, '', url))
    name = cl.text
    
    class_code[name] = code # key : 이름, value : code
    class_url[code] = url # key : code, value : URL
    
code_to_name = dict()

for key, value in class_code.items():
    code_to_name[value] = key

In [6]:
def get_post_basic_info(soup):
    
    num_lst = []
    title_lst = []
    url_lst = []
    
    # soup에서 게시글만 가져오기
    posts = soup.select('#new-board > form > div > table > tbody > tr')
    
    for post in posts:
        if len(post['class']):
            pass
        else:

            num_lst.append(post.select_one('tr > td > span').text) # 글 번호
            url_lst.append(post.select_one('a')['href']) # 글 링크

            post_title = (post.select_one('a').text) # 글 제목
            post_title = re.sub('[\n\t]+','',post_title)
            title_lst.append(re.sub('[ ]{2}','',post_title))
            
    
    return num_lst, title_lst, url_lst

In [150]:
def get_post_detail_info(url_lst, class_code):
    proxy_function = free_proxy.get_proxy_list()
    proxy_idx = 0
    
    def get_post_res(url, fun_p_idx):
        while True:
    
            try:
                proxies = {'http' : 'http://' + proxy_function[fun_p_idx][1], 'https' : 'http://' + proxy_function[fun_p_idx][1]}
                headers = {
                            'user-agent' : ua.random
                            }


                res = requests.get(url, headers = headers, proxies=proxies, timeout=(10,10))
                
                if res.status_code == 200:
                    return res, fun_p_idx
            except Exception as e:
#                 print(e)
                fun_p_idx += 1

                if fun_p_idx == len(proxy_data):
                    fun_p_idx = 0
    
    
    base_url = 'https://www.inven.co.kr/board/lostark/'
    post_detail = []
    for url in tqdm(url_lst):
        res, proxy_idx = get_post_res(url, proxy_idx)

        soup = bs(res.text, 'html.parser')

        # 내용
        post_content = soup.select_one('#tbArticle > div.articleMain > div.articleContent > div #powerbbsContent')
        if post_content:
            post_content = post_content.text 
        else:
            post_content = ''
        
        # 작성일자   
        post_writed_at = soup.select_one('#tbArticle > div.articleHead.hC_silver1 > div > div.articleDate')
        if post_writed_at:
            post_writed_at = post_writed_at.text
        else:
            post_writed_at = ''
        
        articlecode = re.sub(base_url + f"{class_code}/" ,'',url)
        
#         print(articlecode)
        comment_writed_at, comment_content, proxy_function, proxy_idx = get_post_comment(articlecode, proxy_function, proxy_idx, class_code)
        
        post_detail.append({'content' : post_content, 'writed_at' : post_writed_at, 'comments' : comment_content, 'comment_writed_at' : comment_writed_at})
        
    return post_detail

In [149]:
def get_post_comment(articlecode , proxy_data, proxy_idx, class_code):
    
    comment_writed_at = []
    comment_content = []
    
    comment_url = 'https://www.inven.co.kr/common/board/comment.json.php'
    
    while True:
#         print(proxy_idx)
        try:
            proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
            headers = {
            'User-Agent': ua.random,
            'X-Requested-With':'XMLHttpRequest'
            }

            data={
            'comeidx': str(class_code),
            'articlecode': str(articlecode),
            'sortorder': 'date',
            'act': 'list',
            'out': 'json',
            'replyidx': '0',
            }

            cmt_res = requests.post(comment_url, headers = headers, data = data)

            cmt_json = cmt_res.json()

            if cmt_json['cmtcount']:
                for cmt in cmt_res.json()['commentlist'][0]['list']:
                    comment_writed_at.append(cmt['o_date'])
                    content = (cmt['o_comment'])
                    comment_content.append(re.sub('[&amp|nbsp|;]+', ' ', content))
            else:
                comment_content.append('')
                comment_writed_at.append('')
            break
        except Exception as e:
#             print(e)
            proxy_idx += 1

            if proxy_idx == len(proxy_data):
                proxy_data = free_proxy.get_proxy_list()
                proxy_idx = 0
    
    return comment_writed_at, comment_content, proxy_data, proxy_idx

In [9]:
def make_df_lst(num_lst, title_lst, url_lst, detail_info):


    df_lst = []

    for i in range(len(num_lst)):
    #    num_lst, title_lst, url_lst, detail_info

        # 'content', 'writed_at', 'comments', 'comment_writed_at'
        details = detail_info[i]
        content = details['content']
        writed_at = details['writed_at']
        comments = details['comments']
        comment_writed_at = details['comment_writed_at']


        df_lst.append(pd.DataFrame({
            'num' : num_lst[i],
            'title' : title_lst[i],
            'content' : content,
            'writed_at' : writed_at,
            'comments' : [comments],
            'comment_writed_at' : [comment_writed_at],
            'url' : url_lst[i],
        }))

    return df_lst

In [10]:
def check_searched(lst, max_num, min_num):
    
    # 처음과 마지막 사이 글이라면 패스
    go_crawl = False
    for num in lst:
        if (num > max_num) or (num < min_num):
            go_crawl = True
    
    if not go_crawl :
        return 0, 0
    
    # 크롤링 해야하는 최근 게시물
    if max_num in lst:
        last_idx = lst.index(max_num)
    else:
        last_idx = len(lst)
    
    # 크롤링 되지 않은 과거 게시물
    if min_num in lst:
        start_idx = lst.index(min_num)
    else:
        start_idx = 0
        
    return start_idx, last_idx

In [158]:
def concat_df(df, df_lst):

    add_df = pd.concat(df_lst).reset_index(drop=True)
    
    con_df = pd.concat([df, add_df], axis = 0).sort_values('num', ascending=False)

        
    con_df = con_df.drop_duplicates(subset=['title','content']).reset_index(drop=True)

    return con_df

def save_data(df, df_lst, class_code):
    save_df = concat_df(df, df_lst)
    save_df['classcode']  = class_code
    
    path = f'./data/{target_class}.csv'
    
    save_df.to_csv(path, encoding='utf8', index=False)
    print(f'SAVE SUCCESS -> {path}')
    
def load_data(class_code = 0, className = ''):
    
    if class_code != 0:
        name = code_to_name[class_code]
    
    elif className != '':
        name = className
    
    try:
        path = f'./data/{name}.csv'
        df = pd.read_csv(path, encoding='utf8')
    except:
        print(f'Not Exist {name}.csv file')
        df = pd.DataFrame()
    
    return df
    

In [14]:
targets = list(class_code.keys())


In [ ]:
%%time

for target_class in targets[::-1]:
    if target_class == '소울이터': continue
    
    print(f'SEARCHING CLASS : {target_class}')
    
    df = load_data(className = target_class)
    
    proxy_data = free_proxy.get_proxy_list()

    articlecode = class_code[target_class]
    test_url = class_url[class_code[target_class]]

    if len(df):
        searched_max_num = df.num.max()
        searched_min_num = df.num.min()
    else:
        print('Maybe First Searching or Not Loaded Data')
        searched_max_num, searched_min_num = 0, 0

    p_idx = 0
    page = 1
    searching_flag = False

    df_lst = []
    while True:
        print(f'Crawling -> {page}page')
        try:
            proxies = {'http' : 'http://' + proxy_data[p_idx][1], 'https' : 'http://' + proxy_data[p_idx][1]}
            headers = {
                        'user-agent' : ua.random
                        }

            params = {
                'p' : page
            }

            res = requests.get(test_url, headers = headers, params=params, proxies=proxies, timeout=(10,10))

            soup = bs(res.text, 'html.parser')

#             print('get_post_basic_info start')
            num_lst, title_lst, url_lst = get_post_basic_info(soup)
#             print('get_post_basic_info fin')

            num_lst = list(map(int, num_lst))

            s_idx, l_idx = check_searched(num_lst, searched_max_num, searched_min_num)


            num_lst = num_lst[s_idx : l_idx]
            title_lst = title_lst[s_idx : l_idx]
            url_lst = url_lst[s_idx : l_idx]


            if not len(num_lst) : 
                print('Passing Page')
                if page < 5:
                    skip_page = int((searched_max_num-searched_min_num)/65)
                    if skip_page < 5:
                        page +=1
                    else:
                        page += skip_page
                else:
                    page+=1
                continue
            
            end_date = '2023-05-12' # 밸런스패치 내용 공개일
            page_last_article = ''
            
            # 페이지 첫 게시글/ 마지막 게시글 가져오기

            page_articles = soup.select(f'#new-board > form > div > table > tbody > tr')
#             print('articles cnt : ', len(page_articles))
            for idx, at in enumerate(page_articles):
#                 print(at['class'])
                if 'notice' in at['class']:
                    pass
                else:
                    page_articles = page_articles[idx:]
                    break

            page_frist_article = page_articles[1].select_one('td.date').text
            last_idx = len(page_articles) -1
                
            try:                
                while page_last_article == '':
                    page_last_article = page_articles[last_idx].select_one('td.date').text
                    
            except:
                article_last_idx -=1
                

            # 첫 게시글이 end_date보다 작으면 바로 중지
            if ':' in page_frist_article: # 첫 게시글이 시간일 경우(1Page)
                pass
            else:
                page_frist_article = '2023-' + page_frist_article
                if page_frist_article < end_date:
                    print('MODIFY END. 1st Article Date : {:15} / SET END DATE : {}'.format(page_frist_article,end_date))
                    break

            # 첫 게시글에서 중지가 안됐다면 마지막 게시글 확인하고 flag값 변경    
            page_last_article = '2023-' + page_last_article
            if page_last_article < end_date:
                print('MODIFY END. Article Date : {:15} / SET END DATE : {}'.format(page_last_article,end_date))
                searching_flag = True
            
#             print('get_post_detail_info start')
            detail_info = get_post_detail_info(url_lst, articlecode)
#             print('get_post_detail_info fin')

            tmp = make_df_lst(num_lst, title_lst, url_lst, detail_info)

            df_lst+=tmp
            
            # 페이지 서칭 종료
            if searching_flag:
                break
            



            page+=1
        except Exception as e:
            print(e)

            p_idx += 1

            if p_idx == len(proxy_data):
                p_idx = 0
    print("-"*15)
    print('END CLASS({}) ARTICLE IN INVEN'.format(target_class))
    print("-"*15)
    
    if len(df_lst):
        save_data(df, df_lst, articlecode)
    print()
    
    

SEARCHING CLASS : 기상술사
Crawling -> 1page


100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.87s/it]


Crawling -> 2page
Passing Page
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Crawling -> 30page
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [64]:
t = pd.concat(df_lst)

In [67]:
for a in t[-1:]['writed_at']:
    print(a)

2023-05-10 20:21


In [84]:
df

-1

In [154]:
df = concat_df(df, df_lst)

In [164]:
save_data(df, df_lst, target_class)

SAVE SUCCESS -> ./data/호크아이.csv


In [70]:
df = load_data(className =target_class)

In [102]:
((searched_max_num-searched_min_num)/65)

5.492307692307692

False

In [138]:
page_articles[0]['class']

['notice', 'all']

In [140]:
soup = bs(res.text, 'html.parser')
page_articles=soup.select(f'#new-board > form > div > table > tbody > tr')

In [142]:
page_articles[0].select_one('td.date').text

'10-15'

In [136]:
res = requests.get('https://www.inven.co.kr/board/lostark/5343?p=1')